In [9]:
# Phishing Detection API Client
# Simple Jupyter interface to interact with your Flask API

import requests
import json
import time
import os
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from threading import Thread

# Configuration
API_URL = "http://localhost:8050/buoy"  # Change this to your server URL
API_KEY = "Mw0EQeaZRw3yHYGPRlrWzJLUpqhDzG1FqtciykjA0Xo="  # Replace with your actual API key

class PhishingDetector:
    def __init__(self, api_url, api_key):
        self.api_url = api_url
        self.api_key = api_key
        self.headers = {
            "X-API-KEY": api_key,
            "Content-Type": "application/json"
        }
    
    def analyze_text(self, text, model="gemma2:2b"):
        """Send text to API and handle streaming response"""
        payload = {
            "message": text,
            "model": model
        }
        
        try:
            response = requests.post(
                self.api_url,
                headers=self.headers,
                json=payload,
                stream=True,
                timeout=60
            )
            
            if response.status_code != 200:
                return f"Error: HTTP {response.status_code} - {response.text}"
            
            # Process streaming response
            results = []
            for line in response.iter_lines():
                if line:
                    try:
                        line_str = line.decode('utf-8')
                        if line_str.startswith('data: '):
                            data = json.loads(line_str[6:])  # Remove 'data: ' prefix
                            results.append(data)
                    except json.JSONDecodeError:
                        continue
            
            return results
            
        except requests.exceptions.RequestException as e:
            return f"Connection error: {str(e)}"

# Initialize detector
detector = PhishingDetector(API_URL, API_KEY)

def format_response(results):
    """Format the API response for display"""
    if isinstance(results, str):
        return f"<div style='color: red;'>{results}</div>"
    
    html_output = "<div style='font-family: Arial, sans-serif;'>"
    final_result = None
    
    for result in results:
        status = result.get("status", "")
        response = result.get("response", "")
        timestamp = result.get("timestamp", "")
        
        if status == "Done" and isinstance(response, dict):
            final_result = response
            break
        elif status == "AI Analysis" and response:
            html_output += f"<p><strong>🤖 AI Analysis:</strong> {response}</p>"
        elif status and status not in ["Done", "AI Analysis"]:
            html_output += f"<p><em>📝 {status}</em></p>"
    
    # Add final result if available
    if final_result:
        if final_result.get("type") == "chatbot":
            html_output += f"<div style='background: #e8f4f8; padding: 10px; border-radius: 5px; margin: 10px 0;'>"
            html_output += f"<strong>💬 Chatbot Response:</strong><br>{final_result.get('message', '')}"
            html_output += "</div>"
        else:
            url = final_result.get("url", "N/A")
            verdict = final_result.get("predicted_class", "Unknown")
            phishing_conf = final_result.get("confidence_phishing", 0) * 100
            legit_conf = final_result.get("confidence_legit", 0) * 100
            tools_used = final_result.get("tools_used", [])
            
            # Color based on verdict
            color = "#ff4444" if verdict.lower() == "phishing" else "#44ff44" if verdict.lower() == "legit" else "#ffaa44"
            
            html_output += f"<div style='background: {color}22; border-left: 4px solid {color}; padding: 15px; margin: 10px 0;'>"
            html_output += f"<h3 style='margin-top: 0; color: {color};'>🔍 Analysis Result</h3>"
            html_output += f"<p><strong>URL:</strong> {url}</p>"
            html_output += f"<p><strong>Verdict:</strong> <span style='color: {color}; font-weight: bold;'>{verdict.upper()}</span></p>"
            html_output += f"<p><strong>Confidence:</strong> Phishing: {phishing_conf:.1f}% | Legitimate: {legit_conf:.1f}%</p>"
            html_output += f"<p><strong>Tools Used:</strong> {', '.join(tools_used) if tools_used else 'N/A'}</p>"
            html_output += "</div>"
    
    html_output += "</div>"
    return html_output

# Simple function for direct text input
def analyze_text_simple(text):
    """Simple function to analyze text and display results"""
    print("🔄 Analyzing...")
    results = detector.analyze_text(text)
    
    clear_output(wait=True)
    print("✅ Analysis complete!")
    display(HTML(format_response(results)))

# Interactive widget interface
def create_interactive_interface():
    """Create an interactive widget interface"""
    
    # Create widgets
    text_input = widgets.Textarea(
        value='',
        placeholder='Enter your message here (e.g., "Is google.com safe?" or "Check this URL: https://example.com")',
        description='Message:',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    model_dropdown = widgets.Dropdown(
        options=['gemma2:2b', 'gpt-4', 'claude'],
        value='gemma2:2b',
        description='AI Model:',
        disabled=False,
    )
    
    analyze_button = widgets.Button(
        description='🔍 Analyze',
        button_style='primary',
        layout=widgets.Layout(width='150px')
    )
    
    clear_button = widgets.Button(
        description='🗑️ Clear',
        button_style='warning',
        layout=widgets.Layout(width='100px')
    )
    
    output_area = widgets.Output()
    
    # Event handlers
    def on_analyze_click(b):
        with output_area:
            clear_output(wait=True)
            if not text_input.value.strip():
                display(HTML("<div style='color: orange;'>⚠️ Please enter some text to analyze.</div>"))
                return
            
            display(HTML("<div style='color: blue;'>🔄 Analyzing your input...</div>"))
            
            results = detector.analyze_text(text_input.value, model_dropdown.value)
            clear_output(wait=True)
            display(HTML(format_response(results)))
    
    def on_clear_click(b):
        text_input.value = ''
        with output_area:
            clear_output()
    
    analyze_button.on_click(on_analyze_click)
    clear_button.on_click(on_clear_click)
    
    # Layout
    input_box = widgets.VBox([
        text_input,
        widgets.HBox([model_dropdown, analyze_button, clear_button]),
    ])
    
    interface = widgets.VBox([
        widgets.HTML("<h2>🛡️ Phishing Detection Interface</h2>"),
        input_box,
        widgets.HTML("<hr>"),
        widgets.HTML("<h3>📊 Results:</h3>"),
        output_area
    ])
    
    return interface

# Example usage functions
def test_examples():
    """Run some test examples"""
    examples = [
        "Is google.com safe?",
    ]
    
    print("🧪 Running test examples...")
    
    for i, example in enumerate(examples, 1):
        print(f"\n--- Example {i}: {example} ---")
        results = detector.analyze_text(example)
        display(HTML(format_response(results)))
        print("\n" + "="*60)

# Quick setup function
def setup_api_connection():
    """Quick setup to test API connection"""
    global API_KEY, detector
    
    print("🔧 Setting up API connection...")
    
    # Try to get API key from environment or prompt user
    api_key = os.getenv("BUOY_API_KEY")
    if not api_key:
        print("⚠️ API key not found in environment variables.")
        print("Please set your API key:")
        API_KEY = input("Enter your API key: ")
    else:
        API_KEY = api_key
        print("✅ API key loaded from environment")
    
    detector = PhishingDetector(API_URL, API_KEY)
    
    # Test connection
    try:
        test_result = detector.analyze_text("test connection")
        if isinstance(test_result, str) and "error" in test_result.lower():
            print(f"❌ Connection test failed: {test_result}")
        else:
            print("✅ API connection successful!")
    except Exception as e:
        print(f"❌ Connection test failed: {str(e)}")

print("📋 Available functions:")
print("1. setup_api_connection() - Configure and test API connection")
print("2. analyze_text_simple('your text here') - Simple text analysis")
print("3. create_interactive_interface() - Create interactive widget interface")
print("4. test_examples() - Run example tests")
print("\n🚀 Quick start: Run setup_api_connection() first!")

📋 Available functions:
1. setup_api_connection() - Configure and test API connection
2. analyze_text_simple('your text here') - Simple text analysis
3. create_interactive_interface() - Create interactive widget interface
4. test_examples() - Run example tests

🚀 Quick start: Run setup_api_connection() first!


In [10]:
setup_api_connection()  # Automatically run setup on import

🔧 Setting up API connection...
⚠️ API key not found in environment variables.
Please set your API key:
✅ API connection successful!


In [11]:
create_interactive_interface()  # Create the interactive interface
display(create_interactive_interface())  # Display the interface in Jupyter

In [12]:
test_examples()  # Run example tests to demonstrate functionality

🧪 Running test examples...

--- Example 1: Is google.com safe? ---


In [24]:

#%pip install python-whois

import whois

# Example usage of whois
def get_whois_info(domain):
    """Fetch WHOIS information for a given domain."""
    try:
        w = whois.whois(domain)
        return w
    except Exception as e:
        return f"Error fetching WHOIS info: {str(e)}"
# Example domain
domain = "example.com"
whois_info = get_whois_info(domain)
print(f"WHOIS info for {domain}:\n{whois_info}")


WHOIS info for example.com:
Error fetching WHOIS info: 'module' object is not callable
